<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/quiz/Quiz-3.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

| **항목**               | **허용 여부**        | **비고**                                          |
|------------------------|----------------------|---------------------------------------------------|
| **강의노트 참고**      | 허용                 | 수업 중 제공된 강의노트나 본인이 정리한 자료를 참고 가능       |
| **구글 검색**          | 허용                 | 인터넷을 통한 자료 검색 및 정보 확인 가능        |
| **생성 모형 사용**           | 허용 안함            | 인공지능 기반 도구(GPT 등) 사용 불가            |

In [9]:
import datasets
import transformers
import evaluate
import numpy as np
import pandas as pd 
import torch 
from sklearn.model_selection import train_test_split

# `1`. `MBTI` 자료분석 -- 30점

**Kaggle 링크**: [MBTI 데이터셋](https://www.kaggle.com/datasets/datasnaek/mbti-type)

Kaggle에 가입한 후, `archive.zip` 파일을 다운로드하고 Colab에 업로드한 뒤, 아래 명령어를 실행하여 데이터를 불러오라.

```python
!unzip archive.zip
df = pd.read_csv("mbti_1.csv")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
```

이 데이터는 Myers-Briggs Type Indicator(MBTI) 성격 유형과 관련된 사람들의 텍스트 데이터를 포함하고 있다. 데이터는 사람들이 작성한 게시물의 내용과 MBTI 성격 유형 간의 관계를 분석할 수 있도록 구성되어 있다.

- **총 데이터 수**: 약 8600개의 행
  - **학습용 데이터**: 6940개 행  
  - **테스트용 데이터**: 1735개 행

- **각 행의 구성**:
  - **Type (성격 유형)**: MBTI 성격 유형 (예: INTP, ENFJ 등)
  - **Posts (게시물)**: 해당 사용자가 작성한 게시물

`(1)` 주어진 MBTI 데이터셋을 활용하여 **T(Thinking)** 성향의 사람이 작성한 게시물인지 **F(Feeling)** 성향의 사람이 작성한 게시물인지 구분하는 모델을 학습하라. 

**주의**
1. `df_train`을 훈련자료로, `df_test`를 검증자료로 사용하라.
2. 검증자료에 대한 정확도가 80%이상일 경우만 정답으로 인정한다. 

`(2)` 학습된 모형을 사용하여 아래의 Text에 대한 분류를 수행하라. 

In [16]:
["I prefer making decisions based on logic and facts. Analyzing situations objectively is more important to me than considering emotions.", 
 "When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do."]

['I prefer making decisions based on logic and facts. Analyzing situations objectively is more important to me than considering emotions.',
 'When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do.']